In [0]:
!pip3 -qq install torch==0.4.1
!pip -qq install torchtext==0.3.1
!git clone https://github.com/MiuLab/SlotGated-SLU.git
!wget -qq https://raw.githubusercontent.com/yandexdataschool/nlp_course/master/week08_multitask/conlleval.py

In [0]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

if torch.cuda.is_available():
    from torch.cuda import FloatTensor, LongTensor
    DEVICE = torch.device('cuda')
else:
    from torch import FloatTensor, LongTensor
    DEVICE = torch.device('cpu')

np.random.seed(42)

# Диалоговые системы

Диалоговые системы делятся на два типа - *goal-orientied* и *general conversation*.

**General conversation** - это болталка, разговор на свободную тему:  
![](https://i.ibb.co/bFwwGpc/alice.jpg =300x)

Сегодня будем говорить не про них, а про **goal-orientied** системы:
![](https://hsto.org/webt/gj/3y/xl/gj3yxlqbr7ujuqr9r2akacxmkee.jpeg =700x)  
*From [Как устроена Алиса](https://habr.com/company/yandex/blog/349372/)*

Пользователь говорит что-то, это что-то распознается. По распознанному определяется - что, где и когда он хотел. Дальше диалоговый движок решает, действительно ли пользователь знает, чего хотел попросить. Происходит поход в источники - узнать информацию, которую (кажется) запросил пользователь. Исходя из всего этого генерируется некоторый ответ:

![](https://i.ibb.co/8XcdpJ7/goal-orientied.png =900x)  
*From [Как устроена Алиса](https://habr.com/company/yandex/blog/349372/)*

([Клёвая гифка, на которой то же самое, но в динамике](https://raw.githubusercontent.com/yandexdataschool/nlp_course/master/resources/task_oriented_dialog_systems.gif))

Будем учить ту часть, которая посередине - классификатор и теггер. Все остальное обычно - эвристики и захардкоженные ответы.

## Загрузка данных

Есть условно стандартный датасет - atis, который неприлично маленький, на самом деле.

К нему можно взять еще датасет snips - он больше и разнообразнее.

Оба датасета возьмем из репозитория статьи [Slot-Gated Modeling for Joint Slot Filling and Intent Prediction](http://aclweb.org/anthology/N18-2118).

Начнем с atis.

In [0]:
import os 

def read_dataset(path):
    with open(os.path.join(path, 'seq.in')) as f_words, \
            open(os.path.join(path, 'seq.out')) as f_tags, \
            open(os.path.join(path, 'label')) as f_intents:
        
        return [
            (words.strip().split(), tags.strip().split(), intent.strip()) 
            for words, tags, intent in zip(f_words, f_tags, f_intents)
        ]

In [0]:
train_data = read_dataset('SlotGated-SLU/data/atis/train/')
val_data = read_dataset('SlotGated-SLU/data/atis/valid/')
test_data = read_dataset('SlotGated-SLU/data/atis/test/')

In [5]:
intent_to_example = {example[2]: example for example in train_data}
for example in intent_to_example.values():
    print('Intent:\t', example[2])
    print('Text:\t', '\t'.join(example[0]))
    print('Tags:\t', '\t'.join(example[1]))
    print()

Intent:	 atis_flight
Text:	 is	there	a	delta	flight	from	denver	to	san	francisco
Tags:	 O	O	O	B-airline_name	O	O	B-fromloc.city_name	O	B-toloc.city_name	I-toloc.city_name

Intent:	 atis_airfare
Text:	 what	is	the	most	expensive	one	way	fare	from	boston	to	atlanta	on	american	airlines
Tags:	 O	O	O	B-cost_relative	I-cost_relative	B-round_trip	I-round_trip	O	O	B-fromloc.city_name	O	B-toloc.city_name	O	B-airline_name	I-airline_name

Intent:	 atis_airline
Text:	 list	airlines	serving	between	denver	and	san	francisco
Tags:	 O	O	O	O	B-fromloc.city_name	O	B-toloc.city_name	I-toloc.city_name

Intent:	 atis_ground_service
Text:	 tell	me	about	ground	transportation	between	orlando	international	and	orlando
Tags:	 O	O	O	O	O	O	B-fromloc.airport_name	I-fromloc.airport_name	O	B-toloc.city_name

Intent:	 atis_quantity
Text:	 how	many	airlines	have	flights	with	service	class	yn
Tags:	 O	O	O	O	O	O	O	O	B-fare_basis_code

Intent:	 atis_city
Text:	 where	is	lester	pearson	airport
Tags:	 O	O	B-airport_name	

In [6]:
from torchtext.data import Field, LabelField, Example, Dataset, BucketIterator

tokens_field = Field()
tags_field = Field(unk_token=None)
intent_field = LabelField()

fields = [('tokens', tokens_field), ('tags', tags_field), ('intent', intent_field)]

train_dataset = Dataset([Example.fromlist(example, fields) for example in train_data], fields)
val_dataset = Dataset([Example.fromlist(example, fields) for example in val_data], fields)
test_dataset = Dataset([Example.fromlist(example, fields) for example in test_data], fields)

tokens_field.build_vocab(train_dataset)
tags_field.build_vocab(train_dataset)
intent_field.build_vocab(train_dataset)

print('Vocab size =', len(tokens_field.vocab))
print('Tags count =', len(tags_field.vocab))
print('Intents count =', len(intent_field.vocab))

train_iter, val_iter, test_iter = BucketIterator.splits(
    datasets=(train_dataset, val_dataset, test_dataset), batch_sizes=(32, 128, 128), 
    shuffle=True, device=DEVICE, sort=False
)

Vocab size = 869
Tags count = 121
Intents count = 21


## Классификатор интентов

Начнем с классификатора: к какому интенту относится данный запрос.

**Задание** Ничего умного - возьмите rnn'ку и научитесь предсказывать метки-интенты.

In [0]:
class IntentClassifierModel(nn.Module):
    def __init__(self, vocab_size, intents_count, emb_dim=64, lstm_hidden_dim=128, num_layers=1):
        super().__init__()

        self._embed = nn.Embedding(vocab_size, emb_dim)
        self._rnn = nn.LSTM(emb_dim, lstm_hidden_dim, num_layers=num_layers, bidirectional=True)
        self._out = nn.Linear(2 * lstm_hidden_dim * num_layers, intents_count)

    def forward(self, inputs):
        embs = self._embed(inputs)
        _, (outputs, _) = self._rnn(embs, None)
        outputs = torch.cat((outputs[0], outputs[1]), -1)
        outputs = self._out(outputs).squeeze(0)
        return outputs

**Задание** `ModelTrainer` для подсчета лосса и accuracy.

In [0]:
class ModelTrainer():
    def __init__(self, model, criterion, optimizer):
        self.model = model
        self.criterion = criterion
        self.optimizer = optimizer
        
    def on_epoch_begin(self, is_train, name, batches_count):
        """
        Initializes metrics
        """
        self.epoch_loss = 0
        self.correct_count, self.total_count = 0, 0
        self.is_train = is_train
        self.name = name
        self.batches_count = batches_count
        
        self.model.train(is_train)
        
    def on_epoch_end(self):
        """
        Outputs final metrics
        """
        return '{:>5s} Loss = {:.5f}, Accuracy = {:.2%}'.format(
            self.name, self.epoch_loss / self.batches_count, self.correct_count / self.total_count
        )
        
    def on_batch(self, batch):
        """
        Performs forward and (if is_train) backward pass with optimization, updates metrics
        """
        logits = self.model(batch.tokens)
        pred = logits.argmax(-1)
        
        self.correct_count += (pred == batch.intent).float().sum()
        self.total_count += len(batch.intent)
        
        loss = self.criterion(logits, batch.intent)
        
        if self.is_train:
            self.optimizer.zero_grad()
            loss.backward()
            nn.utils.clip_grad_norm_(self.model.parameters(), 1.)
            self.optimizer.step()
        self.epoch_loss += loss.item()
         
        return '{:>5s} Loss = {:.5f}, Accuracy = {:.2%}'.format(
            self.name, loss.item(), self.correct_count / self.total_count
        )

In [0]:
import math
from tqdm import tqdm
tqdm.get_lock().locks = []


def do_epoch(trainer, data_iter, pad_idx, is_train, name=None):
    trainer.on_epoch_begin(is_train, name, batches_count=len(data_iter))
    
    with torch.autograd.set_grad_enabled(is_train):
        with tqdm(total=trainer.batches_count) as progress_bar:
            for i, batch in enumerate(data_iter):
                batch_progress = trainer.on_batch(batch)

                progress_bar.update()
                progress_bar.set_description(batch_progress)
                
            epoch_progress = trainer.on_epoch_end()
            progress_bar.set_description(epoch_progress)
            progress_bar.refresh()

            
def fit(trainer, train_iter, pad_idx, epochs_count=1, val_iter=None):
    best_val_loss = None
    for epoch in range(epochs_count):
        name_prefix = '[{} / {}] '.format(epoch + 1, epochs_count)
        do_epoch(trainer, train_iter, pad_idx, is_train=True, name=name_prefix + 'Train:')
        
        if not val_iter is None:
            do_epoch(trainer, val_iter, pad_idx, is_train=False, name=name_prefix + '  Val:')

In [31]:
model = IntentClassifierModel(vocab_size=len(tokens_field.vocab), intents_count=len(intent_field.vocab)).to(DEVICE)

criterion = nn.CrossEntropyLoss().to(DEVICE)
optimizer = optim.Adam(model.parameters())

trainer = ModelTrainer(model, criterion, optimizer)

fit(trainer, train_iter, pad_idx=None, epochs_count=30, val_iter=test_iter)

[1 / 30] Train: Loss = 0.84167, Accuracy = 80.21%: 100%|██████████| 140/140 [00:03<00:00, 43.26it/s]
[1 / 30]   Val: Loss = 0.69717, Accuracy = 84.10%: 100%|██████████| 7/7 [00:00<00:00, 68.41it/s]
[2 / 30] Train: Loss = 0.32925, Accuracy = 91.47%: 100%|██████████| 140/140 [00:03<00:00, 43.46it/s]
[2 / 30]   Val: Loss = 0.44551, Accuracy = 87.46%: 100%|██████████| 7/7 [00:00<00:00, 68.66it/s]
[3 / 30] Train: Loss = 0.20371, Accuracy = 94.60%: 100%|██████████| 140/140 [00:03<00:00, 42.21it/s]
[3 / 30]   Val: Loss = 0.35791, Accuracy = 89.92%: 100%|██████████| 7/7 [00:00<00:00, 69.17it/s]
[4 / 30] Train: Loss = 0.12584, Accuracy = 96.87%: 100%|██████████| 140/140 [00:03<00:00, 43.84it/s]
[4 / 30]   Val: Loss = 0.32841, Accuracy = 92.27%: 100%|██████████| 7/7 [00:00<00:00, 74.01it/s]
[5 / 30] Train: Loss = 0.08547, Accuracy = 98.10%: 100%|██████████| 140/140 [00:03<00:00, 43.93it/s]
[5 / 30]   Val: Loss = 0.27636, Accuracy = 92.50%: 100%|██████████| 7/7 [00:00<00:00, 73.16it/s]
[6 / 30] T

**Задание** Подсчитайте итоговое качество на тесте.

In [32]:
do_epoch(trainer, test_iter, pad_idx=None, is_train=False, name="Test: ")

Test:  Loss = 0.28450, Accuracy = 94.96%: 100%|██████████| 7/7 [00:00<00:00, 76.59it/s]


## Теггер

![](https://commons.bmstu.wiki/images/0/00/NER1.png)  
*From [NER](https://ru.bmstu.wiki/NER_(Named-Entity_Recognition)*

**Задание** Всё ещё ничего умного - простой теггер, как POS, только NER.

In [0]:
class TokenTaggerModel(nn.Module):
    def __init__(self, vocab_size, tags_count, emb_dim=64, lstm_hidden_dim=128, num_layers=1):
        super().__init__()

        self._embed = nn.Embedding(vocab_size, emb_dim)
        self._rnn = nn.LSTM(emb_dim, lstm_hidden_dim, num_layers=num_layers, bidirectional=True)
        self._out = nn.Linear(2 * lstm_hidden_dim * num_layers, tags_count)

    def forward(self, inputs):
        embs = self._embed(inputs)
        outputs, _ = self._rnn(embs, None)
        outputs = self._out(outputs)
        return outputs

**Задание** Обновите `ModelTrainer`: считать нужно всё те же лосс и accuracy, только теперь немного по-другому.

In [0]:
class TaggerTrainer():
    def __init__(self, model, criterion, optimizer, pad_idx):
        self.model = model
        self.criterion = criterion
        self.optimizer = optimizer
        self.pad_idx = pad_idx
        
    def on_epoch_begin(self, is_train, name, batches_count):
        """
        Initializes metrics
        """
        self.epoch_loss = 0
        self.correct_count, self.total_count = 0, 0
        self.is_train = is_train
        self.name = name
        self.batches_count = batches_count
        
        self.model.train(is_train)
        
    def on_epoch_end(self):
        """
        Outputs final metrics
        """
        return '{:>5s} Loss = {:.5f}, Accuracy = {:.2%}'.format(
            self.name, self.epoch_loss / self.batches_count, self.correct_count / self.total_count
        )
        
    def on_batch(self, batch):
        """
        Performs forward and (if is_train) backward pass with optimization, updates metrics
        """
        target = batch.tags
        logits = self.model(batch.tokens)
        
        mask = (target != self.pad_idx).float()
        pred = logits.argmax(-1)
        
        self.correct_count += ((pred == batch.tags).float() * mask).sum()
        self.total_count += mask.sum()
        
        loss = self.criterion(logits.view(-1, logits.shape[-1]), target.view(-1))
        
        if self.is_train:
            self.optimizer.zero_grad()
            loss.backward()
            nn.utils.clip_grad_norm_(self.model.parameters(), 1.)
            self.optimizer.step()
        self.epoch_loss += loss.item()
         
        return '{:>5s} Loss = {:.5f}, Accuracy = {:.2%}'.format(
            self.name, loss.item(), self.correct_count / self.total_count
        )

In [15]:
pad_idx = tags_field.vocab.stoi['pad']

model = TokenTaggerModel(vocab_size=len(tokens_field.vocab), tags_count=len(tags_field.vocab)).to(DEVICE)

criterion = nn.CrossEntropyLoss(ignore_index=pad_idx).to(DEVICE)
optimizer = optim.Adam(model.parameters())

trainer = TaggerTrainer(model, criterion, optimizer, pad_idx)

fit(trainer, train_iter, pad_idx=pad_idx, epochs_count=30, val_iter=test_iter)

[1 / 30] Train: Loss = 1.40379, Accuracy = 74.70%: 100%|██████████| 140/140 [00:03<00:00, 42.94it/s]
[1 / 30]   Val: Loss = 0.65426, Accuracy = 87.18%: 100%|██████████| 7/7 [00:00<00:00, 68.40it/s]
[2 / 30] Train: Loss = 0.34895, Accuracy = 92.65%: 100%|██████████| 140/140 [00:03<00:00, 43.29it/s]
[2 / 30]   Val: Loss = 0.35030, Accuracy = 92.72%: 100%|██████████| 7/7 [00:00<00:00, 67.49it/s]
[3 / 30] Train: Loss = 0.17930, Accuracy = 96.01%: 100%|██████████| 140/140 [00:03<00:00, 42.96it/s]
[3 / 30]   Val: Loss = 0.24280, Accuracy = 94.64%: 100%|██████████| 7/7 [00:00<00:00, 72.64it/s]
[4 / 30] Train: Loss = 0.11331, Accuracy = 97.58%: 100%|██████████| 140/140 [00:03<00:00, 43.43it/s]
[4 / 30]   Val: Loss = 0.19396, Accuracy = 95.70%: 100%|██████████| 7/7 [00:00<00:00, 69.03it/s]
[5 / 30] Train: Loss = 0.07779, Accuracy = 98.33%: 100%|██████████| 140/140 [00:03<00:00, 43.03it/s]
[5 / 30]   Val: Loss = 0.17261, Accuracy = 96.32%: 100%|██████████| 7/7 [00:00<00:00, 74.45it/s]
[6 / 30] T

NER обычно оценивают по F1-скору угадывания слотов. Для этого все перетаскивают скрипт conlleval друг у друга :)

**Задание** Напишите функцию для оценки теггера.

In [16]:
from conlleval import evaluate

def eval_tagger(model, test_iter):
    true_seqs, pred_seqs = [], []

    model.eval()
    with torch.no_grad():
        for batch in test_iter:
            target = batch.tags
            logits = model(batch.tokens).argmax(-1)
            
            seq_lengths = target.argmin(0)
            seq_lengths[seq_lengths == 0] = batch.tags.size(0)
            
            for seq_ind, seq_len in enumerate(seq_lengths):
                true_seqs.append(' '.join([tags_field.vocab.itos[ind] for ind in target[:seq_len, seq_ind]]))
                pred_seqs.append(' '.join([tags_field.vocab.itos[ind] for ind in logits[:seq_len, seq_ind]]))
            
    print('F1 = {:.2f}%, Precision = {:.2f}%, Recall = {:.2f}%'.format(*evaluate(true_seqs, pred_seqs, verbose=False)))
    
eval_tagger(model, test_iter)

F1 = 93.59%, Precision = 94.06%, Recall = 93.82%


## Multi-task learning

Мы уже обсуждали - multi-task learning крут, моден и молодежен. Давайте ~~будем как он~~ реализуем модель, которая умеет сразу и предсказывать теги и интенты. Идея в том, что в этом всем есть общая информация, которая должна помочь как одной, так и другой задаче: зная интент, можно понять, какие слоты вообще могут быть, а зная слоты, можно угадать и интент.

**Задание** Реализуйте объединенную модель.

In [0]:
class SharedModel(nn.Module):
    def __init__(self, vocab_size, intents_count, tags_count, emb_dim=64, lstm_hidden_dim=128, num_layers=1):
        super().__init__()

        self._intents_embs = nn.Embedding(vocab_size, emb_dim)
        self._tags_embs = nn.Embedding(vocab_size, emb_dim)
        self._rnn = nn.LSTM(2 * emb_dim, lstm_hidden_dim, num_layers=num_layers, bidirectional=True)
        self._intents_out = nn.Linear(2 * lstm_hidden_dim * num_layers, intents_count)
        self._tags_out = nn.Linear(2 * lstm_hidden_dim * num_layers, tags_count)

    def forward(self, inputs):
        intents_embs = self._intents_embs(inputs)
        tags_embs = self._tags_embs(inputs)
        
        embs = torch.cat((intents_embs, tags_embs), -1)
        
        output, (hidden, _) = self._rnn(embs, None)
        
        intents = torch.cat((hidden[0], hidden[1]), -1)
        intents = self._intents_out(intents).squeeze(0)
        
        tags = self._tags_out(output)
        
        return intents, tags

In [0]:
class SharedTrainer():
    def __init__(self, model, intents_criterion, tags_criterion, optimizer, pad_idx):
        self.model = model
        self.intents_criterion = intents_criterion
        self.tags_criterion = tags_criterion
        self.optimizer = optimizer
        self.pad_idx = pad_idx
        
    def on_epoch_begin(self, is_train, name, batches_count):
        """
        Initializes metrics
        """
        self.epoch_loss = 0
        self.intents_correct_count, self.intents_total_count = 0, 0
        self.tags_correct_count, self.tags_total_count = 0, 0
        self.is_train = is_train
        self.name = name
        self.batches_count = batches_count
        
        self.model.train(is_train)
        
    def on_epoch_end(self):
        """
        Outputs final metrics
        """
        return '{:>5s} Loss = {:.5f}, Intents Accuracy = {:.2%}, Tags Accuracy = {:.2%}'.format(
            self.name, self.epoch_loss / self.batches_count,
            self.intents_correct_count / self.intents_total_count,
            self.tags_correct_count / self.tags_total_count
        )
        
    def on_batch(self, batch):
        """
        Performs forward and (if is_train) backward pass with optimization, updates metrics
        """
        intents, tags = self.model(batch.tokens)
        intents_pred = intents.argmax(-1)
        tags_pred = tags.argmax(-1)

        mask = (batch.tags != self.pad_idx).float()

        self.intents_correct_count += (intents_pred == batch.intent).float().sum()
        self.intents_total_count += len(batch.intent)
        self.tags_correct_count += ((tags_pred == batch.tags).float() * mask).sum()
        self.tags_total_count += mask.sum()

        intents_loss = self.intents_criterion(intents, batch.intent)
        tags_loss = self.tags_criterion(tags.view(-1, tags.shape[-1]), batch.tags.view(-1))
        loss = intents_loss + tags_loss

        if self.is_train:
            self.optimizer.zero_grad()
            loss.backward()
            nn.utils.clip_grad_norm_(self.model.parameters(), 1.)
            self.optimizer.step()
        self.epoch_loss += loss.item()

        return '{:>5s} Loss = {:.5f}, Intents Accuracy = {:.2%}, Tags Accuracy = {:.2%}'.format(
            self.name, loss.item(), self.intents_correct_count / self.intents_total_count,
            self.tags_correct_count / self.tags_total_count
        )

In [36]:
pad_idx = tags_field.vocab.stoi['pad']

model = SharedModel(vocab_size=len(tokens_field.vocab),
                    intents_count=len(intent_field.vocab),
                    tags_count=len(tags_field.vocab)).to(DEVICE)

tags_criterion = nn.CrossEntropyLoss(ignore_index=pad_idx).to(DEVICE)
intents_criterion = nn.CrossEntropyLoss().to(DEVICE)
optimizer = optim.Adam(model.parameters())

trainer = SharedTrainer(model, intents_criterion, tags_criterion, optimizer, pad_idx)

fit(trainer, train_iter, pad_idx=pad_idx, epochs_count=30, val_iter=test_iter)

[1 / 30] Train: Loss = 2.24809, Intents Accuracy = 78.70%, Tags Accuracy = 76.08%: 100%|██████████| 140/140 [00:03<00:00, 36.45it/s]
[1 / 30]   Val: Loss = 1.31238, Intents Accuracy = 82.64%, Tags Accuracy = 88.86%: 100%|██████████| 7/7 [00:00<00:00, 63.03it/s]
[2 / 30] Train: Loss = 0.64335, Intents Accuracy = 91.74%, Tags Accuracy = 94.15%: 100%|██████████| 140/140 [00:03<00:00, 35.97it/s]
[2 / 30]   Val: Loss = 0.81819, Intents Accuracy = 88.02%, Tags Accuracy = 93.55%: 100%|██████████| 7/7 [00:00<00:00, 66.80it/s]
[3 / 30] Train: Loss = 0.34548, Intents Accuracy = 95.02%, Tags Accuracy = 96.94%: 100%|██████████| 140/140 [00:03<00:00, 35.98it/s]
[3 / 30]   Val: Loss = 0.62183, Intents Accuracy = 90.03%, Tags Accuracy = 94.97%: 100%|██████████| 7/7 [00:00<00:00, 62.40it/s]
[4 / 30] Train: Loss = 0.21268, Intents Accuracy = 97.25%, Tags Accuracy = 97.99%: 100%|██████████| 140/140 [00:03<00:00, 36.88it/s]
[4 / 30]   Val: Loss = 0.49318, Intents Accuracy = 92.05%, Tags Accuracy = 96.04%

In [37]:
do_epoch(trainer, test_iter, pad_idx=None, is_train=False, name="Test: ")

Test:  Loss = 0.39201, Intents Accuracy = 95.86%, Tags Accuracy = 97.36%: 100%|██████████| 7/7 [00:00<00:00, 65.99it/s]


In [38]:
from conlleval import evaluate

def shared_eval_tagger(model, test_iter):
    true_seqs, pred_seqs = [], []

    model.eval()
    with torch.no_grad():
        for batch in test_iter:
            target = batch.tags
            logits = model(batch.tokens)[1].argmax(-1)
            
            seq_lengths = target.argmin(0)
            seq_lengths[seq_lengths == 0] = batch.tags.size(0)
            
            for seq_ind, seq_len in enumerate(seq_lengths):
                true_seqs.append(' '.join([tags_field.vocab.itos[ind] for ind in target[:seq_len, seq_ind]]))
                pred_seqs.append(' '.join([tags_field.vocab.itos[ind] for ind in logits[:seq_len, seq_ind]]))
            
    print('F1 = {:.2f}%, Precision = {:.2f}%, Recall = {:.2f}%'.format(*evaluate(true_seqs, pred_seqs, verbose=False)))
    
shared_eval_tagger(model, test_iter)

F1 = 94.16%, Precision = 93.19%, Recall = 93.68%


 ## Асинхронное обучение
 
 Вообще, всё затевалось именно из-за этого - асинхронное обучение multi-task модели.
 
Идея описана в статье [A Bi-model based RNN Semantic Frame Parsing Model for Intent Detection and Slot Filling](http://aclweb.org/anthology/N18-2050).

Начнем с такой модели:

![](https://i.ibb.co/N2T1X2f/2018-11-27-2-11-01.png =x400)

Основное отличие от того, что уже реализовали в том, в каком порядке все оптимизируется. Вместо объединенного обучения всех слоев, сети для теггера и для классификатора обучаются отдельно.

На каждом шаге обучения генерируются последовательности скрытых состояний $h^1$ и $h^2$ - для классификатора и для теггера.

Дальше сначала считаются потери от предсказания интента и делается шаг оптимизатора, а затем потери от предсказания теггов - и опять шаг оптимизатора.

**Задание** Реализуйте это.

In [0]:
class AsyncModel(nn.Module):
    def __init__(self, vocab_size, intents_count, tags_count, emb_dim=64, lstm_hidden_dim=128, num_layers=1):
        super().__init__()

        self._embs = nn.Embedding(vocab_size, emb_dim)
        self._intents_rnn = nn.LSTM(emb_dim, lstm_hidden_dim, num_layers=num_layers, bidirectional=True)
        self._tags_rnn = nn.LSTM(emb_dim, lstm_hidden_dim, num_layers=num_layers, bidirectional=True)
        self._intents_transform = nn.Linear(4 * lstm_hidden_dim, 2 * lstm_hidden_dim)
        self._tags_transform = nn.Linear(4 * lstm_hidden_dim + 1, 2 * lstm_hidden_dim)
        self._intents_out = nn.Linear(4 * lstm_hidden_dim, intents_count)
        self._tags_out = nn.Linear(4 * lstm_hidden_dim + 1, tags_count)

    def intents_step(self, inputs, tags_hidden):
        embs = self._embs(inputs)
        intents_hidden, _ = self._intents_rnn(embs, None)
      
        new_intents_hidden = []
        print(intents_hidden.size())
        print(tags_hidden.size())
        for i in range(intents_hidden.size()[0]):
            hidden = torch.cat((intents_hidden[i], tags_hidden[i]), -1)
            new_intents_hidden.append(self._intents_transform(hidden))
        
        hidden = torch.cat((intents_hidden[-1], tags_hidden[-1]), -1)
        intent = self._intents_out(hidden)

        return intent, torch.stack(new_intents_hidden)
    
    def tags_step(self, inputs, intents_hidden, tags):
        embs = self._embs(inputs)
        tags_hidden, _ = self._tags_rnn(embs, None)
        
        new_tags_hidden = []
        new_tags = []
        for i in range(tags_hidden.size()[0]):
            hidden = torch.cat((tags_hidden[i], intents_hidden[i], torch.cat([tags[i]]).unsqueeze(1)), -1)
            new_tags_hidden.append(self._tags_transform(hidden))
            
            hidden = torch.cat((intents_hidden[i], tags_hidden[i], torch.cat([tags[i]]).unsqueeze(1)), -1)
            new_tags.append(self._tags_out(hidden))

        return torch.stack(new_tags), torch.stack(new_tags_hidden)

In [0]:
class AsyncTrainer():
    def __init__(self, model, intents_criterion, tags_criterion, intents_optimizer, tags_optimizer, intent_parameters, tags_parameters, pad_idx):
        self.model = model
        self.intents_criterion = intents_criterion
        self.tags_criterion = tags_criterion
        self.intents_optimizer = intents_optimizer
        self.tags_optimizer = tags_optimizer
        self.intent_parameters = intent_parameters
        self.tags_parameters = tags_parameters
        self.pad_idx = pad_idx
        
    def on_epoch_begin(self, is_train, name, batches_count):
        """
        Initializes metrics
        """
        self.epoch_loss = 0
        self.intents_correct_count, self.intents_total_count = 0, 0
        self.tags_correct_count, self.tags_total_count = 0, 0
        self.is_train = is_train
        self.name = name
        self.batches_count = batches_count
        
        self.tags = None
        self.tags_hidden = None
        self.intents_hidden = None
        
        self.model.train(is_train)
        
    def on_epoch_end(self):
        """
        Outputs final metrics
        """
        return '{:>5s} Loss = {:.5f}, Intents Accuracy = {:.2%}, Tags Accuracy = {:.2%}'.format(
            self.name, self.epoch_loss / self.batches_count,
            self.intents_correct_count / self.intents_total_count,
            self.tags_correct_count / self.tags_total_count
        )
        
    def on_batch(self, batch):
        """
        Performs forward and (if is_train) backward pass with optimization, updates metrics
        """
        if self.tags is None:
            self.tags = FloatTensor().new_zeros(batch.tags.size())
        
        if self.tags_hidden is None:
            self.tags_hidden = self.model._tags_rnn(self.model._embs(batch.tokens), None)[0]
        
        if self.intents_hidden is None:
            self.intents_hidden = self.model._intents_rnn(self.model._embs(batch.tokens), None)[0]
        
        intents, new_intents_hidden = self.model.intents_step(batch.tokens, self.tags_hidden)
        intents_pred = intents.argmax(-1)
        self.intents_correct_count += (intents_pred == batch.intent).float().sum()
        self.intents_total_count += len(batch.intent)
        intents_loss = self.intents_criterion(intents, batch.intent)
        
        if self.is_train:
            self.intents_optimizer.zero_grad()
            intents_loss.backward(retain_graph=True)
            nn.utils.clip_grad_norm_(self.intent_parameters, 1.)
            self.intents_optimizer.step()
        
        mask = (batch.tags != self.pad_idx).float()
        tags, new_tags_hidden = self.model.tags_step(batch.tokens, self.intents_hidden, self.tags)
        tags_pred = tags.argmax(-1)
        self.tags_correct_count += ((tags_pred == batch.tags).float() * mask).sum()
        self.tags_total_count += mask.sum()
        tags_loss = self.tags_criterion(tags.view(-1, tags.shape[-1]), batch.tags.view(-1))
        
        if self.is_train:
            self.tags_optimizer.zero_grad()
            tags_loss.backward(retain_graph=True)
            nn.utils.clip_grad_norm_(self.tags_parameters, 1.)
            self.tags_optimizer.step()
        
        self.epoch_loss += intents_loss.item() + tags_loss.item()
        
        self.tags = tags
        self.tags_hidden = new_tags_hidden
        self.intents_hidden = new_intents_hidden

        return '{:>5s} Loss = {:.5f}, Intents Accuracy = {:.2%}, Tags Accuracy = {:.2%}'.format(
            self.name, intents_loss.item() + tags_loss.item(),
            self.intents_correct_count / self.intents_total_count,
            self.tags_correct_count / self.tags_total_count
        )

Нужно создать отдельные оптимизаторы для каждой части модели.

Отдельные параметры можно получить так:

In [0]:
model = AsyncModel(
    vocab_size=len(tokens_field.vocab),
    intents_count=len(intent_field.vocab),
    tags_count=len(tags_field.vocab)
).to(DEVICE)

tags_parameters = [param for name, param in model.named_parameters() if not name.startswith('_intents')]
intent_parameters = [param for name, param in model.named_parameters() if not name.startswith('_tags')]

Затем их нужно передать в отдельные оптимизаторы и учить отдельно.

*Еще, может быть, пригодится retain_graph параметр метода backward()*.

In [210]:
pad_idx = tags_field.vocab.stoi['pad']

model = AsyncModel(vocab_size=len(tokens_field.vocab),
                   intents_count=len(intent_field.vocab),
                   tags_count=len(tags_field.vocab)).to(DEVICE)

tags_criterion = nn.CrossEntropyLoss(ignore_index=pad_idx).to(DEVICE)
intents_criterion = nn.CrossEntropyLoss().to(DEVICE)
tags_optimizer = optim.Adam(tags_parameters)
intents_optimizer = optim.Adam(intent_parameters)

trainer = AsyncTrainer(model, intents_criterion, tags_criterion, intents_optimizer, tags_optimizer, intent_parameters, tags_parameters, pad_idx)

fit(trainer, train_iter, pad_idx=pad_idx, epochs_count=30, val_iter=test_iter)

[1 / 30] Train: Loss = 7.81341, Intents Accuracy = 0.00%, Tags Accuracy = 0.30%:   1%|          | 1/140 [00:00<00:15,  8.77it/s]


torch.Size([20, 32, 256])
torch.Size([20, 32, 256])
torch.Size([20, 32, 256])
torch.Size([20, 32, 256])


RuntimeError: ignored

In [198]:
<calc intent accuracy and tags F1-score>

SyntaxError: ignored

## Улучшения

**Задание** Посмотрите на параметры в статье и попробуйте добиться похожего качества.

**Задание** Попробуйте заменить корпус, с которым работаете.

### Encoder-decoder

Хорошая идея - использовать не просто независимые предсказания тегов, а декодер над ними:

![](https://i.ibb.co/qrgVSqF/2018-11-27-2-11-17.png =600x)

По сути тут добавляется просто еще слой RNN - на этот раз однонаправленной. При этом его вход в случае предсказания тегов - это предыдущий тег, предыдущее скрытое состояние и скрытые состояния из энкодеров теггов и интента. Для интента - простая RNN.

**Задание** Реализуйте такую модель.

# Async Multi-task Learning for POS Tagging

Это были игрушечные датасеты и не самые хорошие статьи (хоть и с NAACL-2018).

Мне больше нравится вот эта: [Morphosyntactic Tagging with a Meta-BiLSTM Model over Context Sensitive Token Encodings](https://arxiv.org/pdf/1805.08237.pdf). Гораздо больше.

Архитектура там такая:

![](https://i.ibb.co/0nSX6CC/2018-11-27-9-26-15.png =x400)

Multi-task задача - обучение отдельных классификаторов более низкого уровня (над символами и словами) для предсказания тегов отдельными оптимизаторами.

**Задание** Попробовать реализовать, о чем в статье пишется.

# Дополнительные материалы

## Статьи
A Bi-model based RNN Semantic Frame Parsing Model for Intent Detection and Slot Filling, 2018 [[pdf]](http://aclweb.org/anthology/N18-2050)  
Slot-Gated Modeling for Joint Slot Filling and Intent Prediction, 2018 [[pdf]](http://aclweb.org/anthology/N18-2118)  
Morphosyntactic Tagging with a Meta-BiLSTM Model over Context Sensitive Token Encodings, 2018 [[arxiv]](https://arxiv.org/pdf/1805.08237.pdf)

## Блоги
[Как устроена Алиса](https://habr.com/company/yandex/blog/349372/)  

# Сдача

[Форма для сдачи](https://goo.gl/forms/4W7JDuSg3A32Ple72)  
[Feedback](https://goo.gl/forms/9aizSzOUrx7EvGlG3)